In [3]:
import pickle 
import pandas as pd 
import os
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt


In [4]:
import wandb

wandb.login()



Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nickojelly (use `wandb login --relogin` to force relogin)


True

In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [6]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")
    

Running on the GPU


In [7]:
REBUILD_DATA = True
class GRV():
    #class to store training data
    
    #reading data from pickle
    file = open( r"DATA/total_list.npy", 'rb')
    data = pickle.load(file)
    #seperate out classes from inputs
    raceIDs,inputs,classes = zip(*data)
    
    #removing nan from inputs and convert to float
    inputs_df = pd.DataFrame(inputs)
    inputs_df.fillna(value=-1,inplace = True)
    inputs = inputs_df.values.tolist()
    inputs = [[float(i) for i in j] for j in inputs]
    
    
    
    #data
    training_data = []

    def make_training_data(self):
        excluded = 0
        for i in range(len(self.inputs)):
            if len(self.classes[i]) == 8:
                self.training_data.append([np.array(self.inputs[i]),np.array(self.classes[i])])
            else:
                adjustedList = self.classes[i]+([8]*(8-len(self.classes[i])))
                self.training_data.append([np.array(self.inputs[i]),np.array(adjustedList)])
                if len(adjustedList) != 8:
                    print(adjustedList)
        np.save('training_data.npy', self.training_data)
        print("excluded = ", excluded)

if REBUILD_DATA:
    grv = GRV()
    grv.make_training_data()

c:\Users\Nick\.conda\envs\pytorch\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


excluded =  0


In [8]:
#dataset setup
training_data = grv.training_data

X = torch.Tensor([i[0] for i in training_data])
Y = torch.Tensor([i[1] for i in training_data])

#Generate winner only class
Y_w = []
for i in Y:
    n = np.zeros(8)
    index = torch.argmin(i)
    n[index] = float(1)
    Y_w.append(n)
    
Y_w = torch.tensor([i for i in Y_w])
X = X.to(device)
Y_w = Y_w.to(device)

my_dataset = TensorDataset(X,Y_w)
my_dataloader = DataLoader(my_dataset)

C:\Users\Nick\AppData\Local\Temp\ipykernel_20368\3086333344.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:210.)
  X = torch.Tensor([i[0] for i in training_data])


In [9]:
config = dict(
    epochs=50,
    classes=8,
    kernels=[16, 32],
    batch_size=128,
    learning_rate=0.0001,
    validation_split = 0.1,
    dataset="MNIST",
    architecture="CNN",
    dataset_size = len(training_data))

config["dataset_size"]

67233

In [10]:
#have to pass in dataset to get_data (created above)
def make_loader(dataset, config, train=True):
	dataset_size = len(dataset)
	indices = list(range(dataset_size))
	split = int(np.floor(config["validation_split"] * dataset_size))
	random_seed= 42
	np.random.seed(random_seed)
	np.random.shuffle(indices)
	train_indices, val_indices = indices[split:], indices[:split]

	if train:
		dataset_sampler = SubsetRandomSampler(train_indices)
		loader = torch.utils.data.DataLoader(dataset=dataset,
										batch_size=config["batch_size"], 
										shuffle=False,
										pin_memory=False, num_workers=0, sampler = dataset_sampler)
	else:
		dataset_sampler  = SubsetRandomSampler(val_indices)
		loader = torch.utils.data.DataLoader(dataset=dataset,
								shuffle=False,
								pin_memory=False, num_workers=0, sampler = dataset_sampler)


	
	return loader

							

In [11]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(81, 256)
        self.fc2 = nn.Linear(256, 32)
        self.fc3 = nn.Linear(32, 8)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return nn.Softmax(x, dim=1)
    

In [12]:
def build_network(f1_layer_size,f2_layer_size, dropout):
    network = nn.Sequential(  # fully-connected, single hidden layer
        nn.Linear(96, f1_layer_size), nn.ReLU(),
        nn.Linear(f1_layer_size, 8), nn.Softmax())

    # network = nn.Sequential(  # fully-connected, single hidden layer
    #     nn.Linear(96, f1_layer_size), nn.ReLU(),
    #     nn.Linear(f1_layer_size,f2_layer_size), nn.ReLU(),
    #     nn.Linear(f2_layer_size, 8), nn.Softmax())



    return network

In [13]:
def make(config, dataset):
    # Make the data

    train_loader = make_loader(dataset,config, train=True)
    test_loader = make_loader(dataset,config, train=False)
    # Make the model
    #model = Net().to(device)
    model = build_network(config['f1_layer_size'],config['f2_layer_size'],config['dropout'])

    # Make the loss and optimizer
    criterion = nn.HuberLoss(reduction='mean', delta=1.0)
    #criterion = nn.MSELoss()
    #criterion = nn.CrossEntropyLoss()

    optimizer = config["optimizer"]

    if optimizer == "sgd":
        optimizer = optim.SGD(model.parameters(),
                              lr=config["learning_rate"], momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(model.parameters(),
                               lr=config["learning_rate"])
    #optimizer = torch.optim.Adam(
    #    model.parameters(), lr=config["learning_rate"])
    
    return model, train_loader, test_loader, criterion, optimizer

In [14]:
def test(model, test_loader, batch_ct):
    model.eval()
    classL, predL, maxL, correctL = [],[],[],[]
    # Run the model on some test examples
    with torch.no_grad():
        correct, total,max_sum, max_w_sum = 0, 0,0,0
        pred = []
        for images, labels in test_loader:
            images, labels = images, labels
            outputs = model(images)
            #print(f"{images=},\n,{labels=}")
            max, predicted = torch.max(outputs.data, 1)
            _,real = torch.max(labels.data,1)
            predL.append(predicted.item())
            maxL.append(max.item())
            #print(f"{real.item(),predicted.item()=}")
            total += labels.size(0)
            correct += (predicted.item() == real.item())
            correctL.append((predicted.item() == real.item()))
            classL.append(real.item())
            if (predicted.item() == real.item()):
                max_sum += max
            else:
                max_w_sum += max

            #print(f"{correct=}")

        print(f"Accuracy of the model on the {total} " +
              f"test images: {100 * correct / total}%")

        
        wandb.log({"test_accuracy": correct / total, "correct_conf": max_sum/correct, "incorrect_conf":(max_w_sum)/(total-correct)})

        #logdf = pd.DataFrame(data = {'class':classL,'pred':predL,'max':maxL,'correct':correctL})
        #table = wandb.Table(dataframe=logdf)
        #wandb.log({"table_key": table})
        #classCounts = logdf["class"].value_counts()
        #predCounts = logdf["class"].value_counts()
        #boxplot = logdf.boxplot(column=['max'],by='correct')
        
        #print(classCounts, predCounts)
        #boxplot
        #plt.savefig("boxplot.png")
        #wandb.log({"boxplot":boxplot})



    # Save the model in the exchangeable ONNX format
    #torch.onnx.export(model, images, "model.onnx")
    #wandb.save("model.onnx")


In [15]:
def train(model, loader,test_loader, criterion, optimizer, config):
    # Tell wandb to watch what the model gets up to: gradients, weights, and more!
    wandb.watch(model, criterion, log="all", log_freq=10)

    # Run training and track with wandb
    total_batches = len(loader) * config.epochs
    example_ct = 0  # number of examples seen
    batch_ct = 0
    for epoch in tqdm(range(config.epochs)):
        for _, (images, labels) in enumerate(loader):

            loss = train_batch(images, labels, model, optimizer, criterion, btch_count=batch_ct)
            example_ct +=  len(images)
            batch_ct += 1

            # Report metrics every 25th batch
            if ((batch_ct + 1) % 250) == 0:
                train_log(loss, example_ct, epoch)

        if epoch %20 ==0:
            test(model,test_loader, batch_ct)

def train_batch(images, labels, model, optimizer, criterion, btch_count=0):
    images, labels = images, labels
    
    # Forward pass ➡
    outputs = model(images)
    loss = criterion(outputs, labels.float())
    
    # Backward pass ⬅
    optimizer.zero_grad()
    loss.backward()

    # Step with optimizer
    optimizer.step()

    return loss

In [16]:
def train_log(loss, example_ct, epoch):
    # Where the magic happens
    wandb.log({"epoch": epoch, "loss": loss}, step=example_ct)
    #print(f"Loss after " + str(example_ct).zfill(5) + f" examples: {loss:.3f}")

In [17]:
def model_pipeline(config=None):
    dataset = my_dataset
    # tell wandb to get started
    with wandb.init(project="After sweep", config=config):
      # access all HPs through wandb.config, so logging matches execution!
      
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)

      # make the model, data, and optimization problem
      model, train_loader, test_loader, criterion, optimizer = make(config, dataset)
      model = model.to(device)
      print(model)

      # and use them to train the model
      train(model, train_loader,test_loader, criterion, optimizer, config)

      # and test its final performance
      test(model, test_loader)

    return model

In [18]:
sweep_config

NameError: name 'sweep_config' is not defined

In [19]:
CUDA_LAUNCH_BLOCKING=1


normal_config = dict(
    epochs=5000,
    classes=8,
    f1_layer_size=64,
    f2_layer_size=32,
    batch_size=128,
    learning_rate=0.0001,
    optimizer = 'adam',
    loss = 'Huber',
    dropout = 0.2, 
    validation_split = 0.1,
    dataset="MNIST",
    architecture="CNN",
    dataset_size = len(training_data))


model = model_pipeline(config=normal_config)

  0%|          | 0/5000 [00:00<?, ?it/s]

{'epochs': 5000, 'classes': 8, 'f1_layer_size': 64, 'f2_layer_size': 32, 'batch_size': 128, 'learning_rate': 0.0001, 'optimizer': 'adam', 'loss': 'Huber', 'dropout': 0.2, 'validation_split': 0.1, 'dataset': 'MNIST', 'architecture': 'CNN', 'dataset_size': 67233}
5000
{'epochs': 5000, 'classes': 8, 'f1_layer_size': 64, 'f2_layer_size': 32, 'batch_size': 128, 'learning_rate': 0.0001, 'optimizer': 'adam', 'loss': 'Huber', 'dropout': 0.2, 'validation_split': 0.1, 'dataset': 'MNIST', 'architecture': 'CNN', 'dataset_size': 67233}
Sequential(
  (0): Linear(in_features=96, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=8, bias=True)
  (3): Softmax(dim=None)
)


c:\Users\Nick\.conda\envs\pytorch\lib\site-packages\torch\nn\modules\container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
  0%|          | 1/5000 [00:14<19:31:12, 14.06s/it]

Accuracy of the model on the 6723 test images: 13.77361297040012%


  0%|          | 21/5000 [00:52<5:41:31,  4.12s/it]

...

 64%|██████▍   | 3221/5000 [1:44:24<2:17:34,  4.64s/it]

Accuracy of the model on the 6723 test images: 25.464822251970848%


 65%|██████▍   | 3241/5000 [1:45:12<2:23:59,  4.91s/it]

Accuracy of the model on the 6723 test images: 25.286330507214043%


 65%|██████▍   | 3243/5000 [1:45:18<1:52:57,  3.86s/it]